## Ejecicio 4 DS
Definición de clases:

In [2]:
from enum import Enum
from abc import ABC, abstractmethod


class EstadoMotor(Enum):
    APAGADO = 0
    ACELERANDO = 1
    FRENANDO = 2
    ENCENDIDO = 3

class Filtro(ABC):
    @abstractmethod
    def ejecutar(self, revoluciones, estado_motor):
        pass
    
class CalcularVelocidad(Filtro):
    def __init__(self):
        pass
    
    def ejecutar(self, revoluciones, estado_motor):
        incremento_velocidad = 0
        
        if estado_motor == EstadoMotor.ACELERANDO:
            incremento_velocidad = 100
        elif estado_motor == EstadoMotor.FRENANDO:
            incremento_velocidad = -100
        
        # Aplicar incremento de velocidad
        revoluciones += incremento_velocidad
        
        # Limitar a 5000 RPM
        revoluciones = min(revoluciones, 5000)
        
        return revoluciones

class RepercutirRozamiento(Filtro):
    def __init__(self):
        pass
    
    def ejecutar(self, revoluciones, estado_motor):
        # Disminuir las revoluciones debido al rozamiento
        return revoluciones - 1

class Objetivo:
    def ejecutar(self, revoluciones, estado_motor):
        # Lógica para mostrar parámetros del movimiento en el salpicadero
        pass

class CadenaFiltros:
    def __init__(self):
        self.filtros = []

    def agregar_filtro(self, filtro):
        self.filtros.append(filtro)

    def ejecutar_filtros(self, revoluciones, estado_motor):
        for filtro in self.filtros:
            revoluciones = filtro.ejecutar(revoluciones, estado_motor)
        return revoluciones

class GestorFiltros:
    def __init__(self):
        self.cadena_filtros = CadenaFiltros()

    def agregar_filtro(self, filtro):
        self.cadena_filtros.agregar_filtro(filtro)

    def ejecutar_peticion_filtros(self, revoluciones, estado_motor):
        # Ejecutar filtros y devolver revoluciones modificadas
        return self.cadena_filtros.ejecutar_filtros(revoluciones, estado_motor)
    
class Cliente:
    def __init__(self, gestor_filtros):
        self.gestor_filtros = gestor_filtros

    def enviar_peticion(self, revoluciones, estado_motor):
        return self.gestor_filtros.ejecutar_peticion_filtros(revoluciones, estado_motor)

class Objetivo:
    def __init__(self):
        # Puedes inicializar atributos o realizar otras acciones de inicialización aquí
        pass

    def ejecutar(self, revoluciones, estado_motor):
        # Lógica para mostrar parámetros del movimiento en el salpicadero
        pass

In [6]:
import tkinter as tk

# Aquí deberían ir las definiciones de las clases EstadoMotor, Filtro, CalcularVelocidad, RepercutirRozamiento
# que proporcionaste previamente

class Salpicadero(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.estado_motor = EstadoMotor.APAGADO
        self.create_widgets()

    def create_widgets(self):
        self.label_estado = tk.Label(self, text="APAGADO", font=("Arial", 16))
        self.label_estado.grid(row=0, column=0, columnspan=3, padx=10, pady=10)

        self.btn_encender = tk.Button(self, text="Encender", command=self.toggle_encender)
        self.btn_encender.grid(row=1, column=0, padx=5, pady=5)

        self.btn_acelerar = tk.Button(self, text="Acelerar", command=self.acelerar, state=tk.DISABLED)
        self.btn_acelerar.grid(row=1, column=1, padx=5, pady=5)

        self.btn_frenar = tk.Button(self, text="Frenar", command=self.frenar, state=tk.DISABLED)
        self.btn_frenar.grid(row=1, column=2, padx=5, pady=5)

    def toggle_encender(self):
        if self.estado_motor == EstadoMotor.APAGADO:
            self.estado_motor = EstadoMotor.ENCENDIDO
            self.label_estado.config(text="ENCENDIDO")
            self.btn_acelerar.config(state=tk.NORMAL)
            self.btn_frenar.config(state=tk.NORMAL)
            self.btn_encender.config(text="Apagar")
        else:
            self.estado_motor = EstadoMotor.APAGADO
            self.label_estado.config(text="APAGADO")
            self.btn_acelerar.config(state=tk.DISABLED)
            self.btn_frenar.config(state=tk.DISABLED)
            self.btn_encender.config(text="Encender")

    def acelerar(self):
        if self.estado_motor == EstadoMotor.ENCENDIDO:
            self.label_estado.config(text="ACELERANDO")

    def frenar(self):
        if self.estado_motor == EstadoMotor.ENCENDIDO:
            self.label_estado.config(text="FRENANDO")

class Velocimetro(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.create_widgets()

    def create_widgets(self):
        self.label_velocidad = tk.Label(self, text="Velocidad: 0 km/h", font=("Arial", 12))
        self.label_velocidad.pack(pady=5)

    def actualizar_velocidad(self, velocidad):
        self.label_velocidad.config(text=f"Velocidad: {velocidad} km/h")

class CuentaKilometros(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.create_widgets()
        self.km_total = 0

    def create_widgets(self):
        self.label_km_reciente = tk.Label(self, text="KM reciente: 0 km", font=("Arial", 12))
        self.label_km_reciente.pack(pady=5)
        self.label_km_total = tk.Label(self, text="KM total: 0 km", font=("Arial", 12))
        self.label_km_total.pack(pady=5)

    def actualizar_km(self, distancia):
        self.km_total += distancia
        self.label_km_reciente.config(text=f"KM reciente: {distancia} km")
        self.label_km_total.config(text=f"KM total: {self.km_total} km")

class CuentaRevoluciones(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.create_widgets()

    def create_widgets(self):
        self.label_revoluciones = tk.Label(self, text="Revoluciones: 0 RPM", font=("Arial", 12))
        self.label_revoluciones.pack(pady=5)

    def actualizar_revoluciones(self, rpm):
        self.label_revoluciones.config(text=f"Revoluciones: {rpm} RPM")

class SalpicaderoCompleto(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.create_widgets()

    def create_widgets(self):
        self.salpicadero = Salpicadero(self)
        self.salpicadero.grid(row=0, column=0, padx=10, pady=10)

        self.velocimetro = Velocimetro(self)
        self.velocimetro.grid(row=0, column=1, padx=10, pady=10)

        self.cuenta_km = CuentaKilometros(self)
        self.cuenta_km.grid(row=1, column=0, padx=10, pady=10)

        self.cuenta_revoluciones = CuentaRevoluciones(self)
        self.cuenta_revoluciones.grid(row=1, column=1, padx=10, pady=10)

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Salpicadero Completo")

    salpicadero_completo = SalpicaderoCompleto(master=root)
    salpicadero_completo.pack()

    root.mainloop()
